### Build a Simple LLM Application with LCEL
In this quickstart we'll show you how to build a simple LLM application with LangChain. This application will translate text from English into another language. This is a relatively simple LLM application - it's just a single LLM call plus some prompting. Still, this is a great way to get started with LangChain - a lot of features can be built with just some prompting and an LLM call!

After seeing this video, you'll have a high level overview of:

- Using language models

- Using PromptTemplates and OutputParsers

- Using LangChain Expression Language (LCEL) to chain components together

- Debugging and tracing your application using LangSmith

- Deploying your application with LangServe

In [12]:
import os
from dotenv import load_dotenv

from langchain_groq import ChatGroq


In [14]:
# load_dotenv()

# grop_api_key = os.getenv("GROQ_API_KEY")
# grop_api_key

## Model Downlaod


In [15]:
model = ChatGroq(model='gemma2-9b-it', api_key=grop_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x114395ff0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x114396c80>, model_name='gemma2-9b-it', groq_api_key=SecretStr('**********'))

In [16]:
model.invoke("what are job profiles for LLm in data science")

AIMessage(content='LLMs (Large Language Models) are revolutionizing the field of data science, opening up exciting new job profiles. Here are some examples:\n\n**Directly Leveraging LLMs:**\n\n* **AI Data Scientist:** This role focuses on building and deploying AI models, including those powered by LLMs. Tasks include data preprocessing, feature engineering, model training, evaluation, and deployment.\n* **Prompt Engineer:**  A specialized role focused on crafting effective prompts for LLMs to elicit desired outputs. They understand the nuances of language and how to guide the model towards specific tasks and results.\n* **LLM Trainer/Fine-Tuner:**  These professionals specialize in training and fine-tuning LLMs on specific datasets for specialized applications. They have expertise in machine learning algorithms, data augmentation techniques, and model optimization.\n\n**Indirectly Influenced by LLMs:**\n\n* **Data Analyst:** LLMs can automate data cleaning, summarization, and visualiz

In [6]:
from langchain_core.messages import HumanMessage, AIMessage , SystemMessage


prompt_template = [
        SystemMessage(content="you are expert real state agent and proffessional in this"),
        HumanMessage(content="how to calculate builtup are when efficiency and carpet is give provide formula")
]

prompt_template

[SystemMessage(content='you are expert real state agent and proffessional in this'),
 HumanMessage(content='how to calculate builtup are when efficiency and carpet is give provide formula')]

In [7]:
response = model.invoke(prompt_template)

NameError: name 'model' is not defined

In [8]:
print(response.content)

NameError: name 'response' is not defined

In [9]:
from langchain_core.output_parsers import StrOutputParser

output = StrOutputParser()

output.invoke(response)

NameError: name 'response' is not defined

In [10]:
chain = model|output

NameError: name 'model' is not defined

In [15]:
chain.invoke(prompt_template)

"You're asking about calculating built-up area when you're given the efficiency and carpet area. Here's the breakdown:\n\n**Understanding the Terms**\n\n* **Carpet Area:** This is the usable floor area within the walls of a property, excluding balconies, external walls, and internal walls. \n* **Built-up Area:** This is the total covered area of a property, including the carpet area, balconies, external walls, and internal walls.\n* **Efficiency:** This is the ratio of the built-up area to the carpet area. It's expressed as a percentage.\n\n**Formula**\n\n**Built-up Area = Carpet Area * (100 / Efficiency)**\n\n**Example**\n\nLet's say you have a property with:\n\n* Carpet Area: 1000 square feet\n* Efficiency: 80%\n\n**Calculation:**\n\nBuilt-up Area = 1000 square feet * (100 / 80) = 1250 square feet\n\n**Important Considerations**\n\n* **Local Standards:**  Building codes and regulations can vary by location.  Make sure you're using the correct definition of built-up area and efficienc

## New Prompt template

In [17]:
### Prompt Templates

from langchain_core.prompts import ChatPromptTemplate

generic_prompt = "translate the following text into {language}"

prompt = ChatPromptTemplate.from_messages (
    [("system" , generic_prompt),("user","{text}")]
)

In [19]:
result = prompt.invoke({"language":"hindi","text":"Hello my name is Sachin"})
result

ChatPromptValue(messages=[SystemMessage(content='translate the following text into hindi'), HumanMessage(content='Hello my name is Sachin')])

In [20]:
result.to_messages()

[SystemMessage(content='translate the following text into hindi'),
 HumanMessage(content='Hello my name is Sachin')]

In [21]:
chain = prompt|model|output

In [22]:
chain.invoke({"language":"French","text":"Hello my name is Sachin"})

"Bonjour, je m'appelle Sachin. \n"

In [23]:

generic_prompt1 = "translate the following text into {language} based on {text}"
prompt2 =ChatPromptTemplate.from_template(generic_prompt1)
prompt2

ChatPromptTemplate(input_variables=['language', 'text'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['language', 'text'], template='translate the following text into {language} based on {text}'))])

In [25]:
result = prompt2.invoke({"language":"French","text":"Hello my name is Sachin"})
result.to_messages()


[HumanMessage(content='translate the following text into French based on Hello my name is Sachin')]

In [26]:
chain2 = prompt2|model|output

In [31]:
chain2.invoke({"language":"english","text":"पुनरावलोकन > भाषांतर > भाषांतर निवड निवडा. भाषांतर पाहण्यासाठी तुमची भाषा निवडा. घाला निवडा . भाषांतरित मजकूर तुम्ही चरण 1 मध्ये हायलाइट केलेल्या मजकुराची जागा घेईल."})

"**Review > Translation > Select Translation. Select your language to view the translation.  Choose.**\n\n**The translated content will replace the highlighted content from Step 1.** \n\n\nLet me know if you'd like me to translate anything else! \n"

In [33]:
chennai = """01-04-2012 அå² உயßÚதÜபØட லÚå வகாØ} மÜ®,
09-06-2017 அå² ¯தà ஒேர ராக 33 சத தÝ ¤ைறÔகÜபØட«.
ேம³Ý, äபைன, நåெகாைட, பமாäறÝ மä²Ý ¤©Ýப
உ²Üனßக´Ô¤ இைடேய அàலாத ஏäபா© ஆவணÕக´Ôகான
ப¶Ô கØடணÝ ஒ± சத தÚ±Û« நாå¤ சத தமாக
உயßÚதÜபØட«. சÛைத மÜä¤ ஏäப வகாØ} மÜைப
உயßÚத¶Ý, ப¶Ô கØடணÚைதÔ ¤ைறÔக¶Ý பàேவ²
தரÜனட±Û« ெதாடßÛ« ேகாÔைககã வÛத வÙணÝ
உãளன. இÛத ேகாÔைககைள ஏä², வகாØ} மÜà
±ÚதÚைதÜ பÛ«ைரÔக ஒ± ¤µைவ அர¦ அைமÚ«ãள«.
இÔ¤µ லஅளைவ எÙ வாயாக ±ÚதÝ ேமäெகாãள கால"""

In [35]:
cehnnai = chain2.invoke({"language":"english","text":chennai})

In [37]:
print(cehnnai)

On 01-04-2012, a directive was issued stating that applications for University admissions should be made through online mode only.

On 09-06-2017, an amendment was made stating that 33% of the seats should be reserved for the government quota.

Hence, the counseling for admission to universities will be conducted considering the government quota of 33% seats.

Applications received from students belonging to the backward classes, minority communities, and other categories will be considered for admission based on the available seats. 

The date for the counseling process will be announced later. We request all students who have applied for admission to note this information and wait for further updates.



Please let me know if you need any further clarification or translation! 

